# Trabalhando com Dados de Séries Temporais em Python

## O que são Dados de Séries Temporais?

In [ ]:
import pandas as pd

df = pd.read_csv("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo&datatype=csv")

df.head() 

In [ ]:
df['timestamp'] = pd.to_datetime(df.timestamp)

df.head()

In [ ]:
df.index = df.timestamp

print(df.head())

df['close'].plot()

### Podemos fazer tudo isso de uma só vez com...

In [ ]:
one_step = pd.read_csv("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo&datatype=csv" \
                       , parse_dates=['timestamp'], index_col = 'timestamp')

one_step.head()

# Trabalhando com Dados de Séries Temporais

## Indexação e fatiamento baseados em tempo

In [ ]:
# Podemos fatiar o dataframe

start = df['timestamp'].iloc[10]
stop = df['timestamp'].iloc[20]

sample = df[start:stop]

sample.head()

In [ ]:
# Podemos acessar valores de data e hora

df.timestamp.dt.month

## O que é Estacionariedade? Por que isso importa?

Uma série temporal **estacionária** é aquela cujas propriedades estatísticas, como média, variância e autocorrelação, não mudam ao longo do tempo.

- **Média Constante:** A média da série não deve ser uma função do tempo.

- **Variância Constante:** A variância da série não deve ser uma função do tempo. Essa propriedade é conhecida como homocedasticidade.

- **Covariância Constante:** A covariância do i-ésimo termo e do (i+m)-ésimo termo não deve ser uma função do tempo.

### A história de dois conjuntos de dados

In [ ]:
### Dados de Frequência Cardíaca


hr = pd.read_csv("https://raw.githubusercontent.com/alyssaq/python-data-science-intro/master/datasets/heart-rate-time-series.csv", header=None)
hr.columns = ['rate']

# Plotar os dados de frequência cardíaca
hr.plot(title='HR Data')

### A história de dois conjuntos de dados

In [ ]:
df['close'].plot(title='Stock Data')

## Testes de Estacionariedade

### Teste Dickey-Fuller Aumentado (ADF)

O teste ADF é um dos testes estatísticos mais populares para verificar a estacionariedade de uma série temporal. A hipótese nula do teste ADF é que a série temporal é não estacionária devido à presença de uma raiz unitária.

$ 
\Delta Y_t = \alpha + \beta t + \gamma Y_{t-1} + \delta_1 \Delta Y_{t-1} + \ldots + \delta_{p-1} \Delta Y_{t-p+1} + \epsilon_t
$

Onde:
- $\Delta Y_t$ é a série de diferenças
- $\alpha$, $\beta$, and $\gamma$ são coeficientes
- $p$ é o número de defasagens
- $\epsilon_t$ é o termo de erro



### Teste de Kwiatkowski-Phillips-Schmidt-Shin (KPSS)

O teste KPSS é outro teste popular para estacionariedade, em que a hipótese nula é que os dados são estacionários em torno de uma tendência determinística.

$
Y_t = \alpha + \beta t + \rho Y_{t-1} + \epsilon_t
$

Onde:
- $Y_t$ é a série temporal
- $\alpha$ and $\beta$ são coeficientes
- $\rho$ é o parâmetro autorregressivo
- $\epsilon_t$ é o termo de erro

## Os testes em ação

### O teste ADF

In [ ]:
from statsmodels.tsa.stattools import adfuller

result_adf_stock = adfuller(df['close'])
result_adf_hr = adfuller(hr)

print(f'ADF Statistic Stock: {result_adf_stock[0]}')
print(f'p-value: {result_adf_stock[1]}')
print('Critical Values:')
for key, value in result_adf_stock[4].items():
    print(f'\t{key}: {value}')

print("---------------------------")
    
print(f'ADF Statistic HR: {result_adf_hr[0]}')
print(f'p-value: {result_adf_hr[1]}')
print('Critical Values:')
for key, value in result_adf_hr[4].items():
    print(f'\t{key}: {value}') 

## Os testes em ação

### O teste KPSS

In [ ]:
### NOTE: Você pode não obter os mesmos valores do vídeo.
## Os dados são ao vivo, o que resultará em uma alteração no valor a cada execução. No entanto,
## a conclusão é a mesma.

from statsmodels.tsa.stattools import kpss

# Executar teste KPSS
result_adf_stock = kpss(df['close'], nlags="auto")
result_adf_hr = kpss(hr, nlags="auto")


print(f'KPSS Statistic: {result_adf_stock[0]}')
print(f'p-value: {result_adf_stock[1]}')
print('Critical Values:')
for key, value in result_adf_stock[3].items():
    print(f'\t{key}: {value}')
    
print("---------------------------")

print(f'KPSS Statistic: {result_adf_hr[0]}')
print(f'p-value: {result_adf_hr[1]}')
print('Critical Values:')
for key, value in result_adf_hr[3].items():
    print(f'\t{key}: {value}')

## Como corrigimos dados de TS que não são Estacionários?

## Diferenciação

A diferenciação é uma técnica usada para tornar estacionária uma série temporal não estacionária. Ela envolve a transformação da série calculando as diferenças entre observações consecutivas.

### Diferenciação de Primeira Ordem

A primeira diferença é dada pela seguinte equação:

$ 
\Delta Y_t = Y_t - Y_{t-1}
$


In [ ]:
df = df.sort_index()
new_station_data = df['close'].diff()
new_station_data = new_station_data.dropna()
new_station_data.head()

In [ ]:
### NOTE: Você pode não obter os mesmos valores do vídeo.
## Os dados são dinâmicos, o que resultará em uma alteração no valor a cada execução. No entanto,
## a conclusão é a mesma.

result_adf_stock = adfuller(new_station_data)
result_kpss_stock = kpss(new_station_data, nlags="auto")

print(f'ADF Statistic Stock: {result_adf_stock[0]}')
print(f'p-value: {result_adf_stock[1]}')
print('Critical Values:')
for key, value in result_adf_stock[4].items():
    print(f'\t{key}: {value}')
    
print("---------------------------")

print(f'KPSS Statistic: {result_kpss_stock[0]}')
print(f'p-value: {result_kpss_stock[1]}')
print('Critical Values:')
for key, value in result_kpss_stock[3].items():
    print(f'\t{key}: {value}')
    


In [ ]:
new_station_data.plot(title='Stock Data')